In [1]:
# Import libraries
import requests
import csv
import pandas as pd

In [2]:
# BSCScan API Setup
def call_api(user, originalHash, symbol, MIN, MAX):
    URL = "https://api.bscscan.com"
    API_KEY = "W9KUR7P8P2HU1R589IS48X4WVJV5B6Z8XC"
    START_BLOCK = "7906054"
    END_BLOCK = "7915038"
    USER_ADDRESS = user

    endpoint = "{}/api?module=account&action=tokentx&address={}&startblock={}&endblock={}&sort=asc&apikey={}".format(URL, USER_ADDRESS, START_BLOCK, END_BLOCK, API_KEY)
    response = requests.get(endpoint)
    JSON_DATA_RAW = response.json()
    
    # Check if user has any transactions in the time frame
    if(JSON_DATA_RAW["status"]=="1"):
        # Iterate over transactions
        for transaction in JSON_DATA_RAW["result"]:
            
            # Check for token symbol
            if(transaction["tokenSymbol"] == symbol):
                value = int(transaction['value']) / pow(10, int(transaction["tokenDecimal"]))
                
                # Check if value is same
                if(int(value) in range(MIN, MAX)):
#                     print("Same token, same value, {}, Original Hash: {}".format(transaction['hash'], originalHash))
                    return transaction['hash']

                # Values not same
                else:
#                     print("Same token, different value, {}, Original Hash: {}".format(transaction['hash'], originalHash))
                    return 0
            
            # Different token
            else:
#                 print("Different token, {}".format(transaction['hash']))
                return 0
    
    # No transactions by the user in the time frame
    else:
#         print("No transaction")
        return 0

In [3]:
df = pd.read_csv('maticbridge.csv')
df["value"] = df["value"].astype(float)
df['ValueDecimal'] = df['value'] / pow(10, df["tokenDecimal"])
df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,ValueDecimal
0,12546134,1622517429,0xed44f1d553920ba941356e2ea5bec716561d2b72e0f0...,2063,0xfdf4b2fd7ad737f3812bd018675df1f289fe18cb417d...,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x5583bbda575f142993c10a335daf29dc7510e8f2,5.000000e+08,USD Coin,USDC,6,23,272350,28000000000,256470,1411942,deprecated,2137,500.000000
1,12546135,1622517435,0x394ccda1c0abd5f4189bd3b9bbe3a1a44fb5fcdcd14d...,1,0x5af8a8fc80b3afc9a934acdafbcdf5cefb1d00bceb7e...,0x34400b1418bde0a4f80d7905d14ed39af8d76e79,0xdac17f958d2ee523a2206206994597c13d831ec7,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,7.217824e+08,Tether USD,USDT,6,74,108181,19715610000,85712,6408537,deprecated,2136,721.782445
2,12546135,1622517435,0xa190bf069d850a88a2336d25eabe5532096e408faddb...,435,0x5af8a8fc80b3afc9a934acdafbcdf5cefb1d00bceb7e...,0xddbbf84382db98573bcc656fa7c6ac8b52466c8b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,1.304566e+20,SushiToken,SUSHI,18,112,102323,19000000000,79854,9656025,deprecated,2136,130.456595
3,12546135,1622517435,0x273b4f81dc161bcfd3c0332d35bf3e340146c81d724c...,5,0x5af8a8fc80b3afc9a934acdafbcdf5cefb1d00bceb7e...,0xe60d8c59b0ac3deb73efa0e84747c48038885898,0xdac17f958d2ee523a2206206994597c13d831ec7,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,1.024557e+09,Tether USD,USDT,6,131,108181,19000000000,85712,12364809,deprecated,2136,1024.556988
4,12546145,1622517568,0xa270ff5e0d43a20e567973fb6fd28d6294f25d16406f...,504,0x8d1f9ec7383c53c0a073607f5f1e0cfc6f3d9244c45c...,0x1b200c7163826a363b5531016a4f0bc8b0277453,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,2.896694e+11,USD Coin,USDC,6,122,114687,23500000000,107218,8693140,deprecated,2126,289669.400000


In [4]:
columnList = list(df.columns)
columnList.append("BSC Hash")
columnList.append("Type")
filteredDf = pd.DataFrame(columns = columnList)

for index, row in df.head(200).iterrows():
    USER_ADDRESS = ""
    TYPE = ""
    FLAG = False
    if(row["from"]=="0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf"):
        USER_ADDRESS = row["to"]
        TYPE = "OUT" # Out of Polygon
    else:
        USER_ADDRESS = row["from"]
        TYPE = "IN" # Into Polygon
    
    MIN = row["ValueDecimal"] - 100
    MAX = row["ValueDecimal"] + 100
    
    if(call_api(USER_ADDRESS, row['hash'], row['tokenSymbol'], int(MIN), int(MAX))!=0):
        print(index)
        filteredDf.loc[index] = df.iloc[index]
        filteredDf.loc[index, 'BSC Hash'] = call_api(USER_ADDRESS, row['hash'], row['tokenSymbol'], int(MIN), int(MAX))
        if(TYPE == "IN"):
            filteredDf.loc[index, 'Destination'] = "Polygon"
        elif(TYPE == "OUT"):
            filteredDf.loc[index, 'Destination'] = "BSC"

92
97
173


In [5]:
filteredDf

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,ValueDecimal,BSC Hash,Type,Destination
92,12546298,1622519698,0xc80c7e4667a9703599fa20262d09784f6c9eb97f02f6...,15,0x8e7f7012061273c0fdf3968ca87863e9d62742d2f7b8...,0x46aaa27dbc0c4d4d1c8afb81e7e032d44e0fde36,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,1.210558e+10,USD Coin,...,114687,25000000000,92218,6575803,deprecated,1973,12105.584884,0x04cc20070cf9aea1070b2c074330097674476ce028d6...,NaN,Polygon
97,12546301,1622519743,0x70d3abf8915080f00964e6293319283f622f5da8f199...,16,0x779d253270c04d728dafb26209810156a9af36da3394...,0x95c2ec97903c993efbe4630f1535282a12fead98,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,4.655000e+09,USD Coin,...,114687,20000000000,92218,11747207,deprecated,1970,4655.000000,0x9926c941d4d713e88fb995f0bf600c88863fc51b4018...,NaN,Polygon
173,12546394,1622520937,0x4826eca2a9e918590b397eb239773a1ab36593c3d1f7...,1,0xf1990605b7dc33681d19cbd8ff61e457e2848da4a8d9...,0x74f40d36f515ae1849e46d900648565b0bda0841,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x40ec5b33f54e0e8a33a975908c5ba1c14e5bbbdf,3.969048e+09,USD Coin,...,114675,17000000000,92206,2873709,deprecated,1877,3969.047721,0xcea76fbf6d5c50f067e092085f41e7ebb2fbb9d8b1d2...,NaN,Polygon


In [13]:
filteredDf.to_csv('filteredDf.csv')